In [1]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import numpy as np

import pandas as pd

/var/folders/bg/xs82ss7x45n2y6rrty58fjd40000gn/T/ipykernel_17792/2162427038.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Replace with your path to one of the subjects from Action-Net
emg_annotations = pd.read_pickle("emg_annotations/S04_1.pkl")

In [7]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt

num_samples = 20

def rectify_signal(data):
    return np.abs(data)

def low_pass_filter(data, fs, cutoff):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(4, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data)

def preprocess_row(row, start_time=0, duration=10, num_samples=100, side="left", num_channels=8):
    timestamps = row[f'myo_{side}_timestamps']
    records = row[f'myo_{side}_readings']
    
    # Pre-processing steps
    # 1. Rectify the signal
    rectified_data = rectify_signal(records)
    rectified_data = rectified_data.T

    # 2. Low pass filter 5 Hz
    fs = 160
    cutoff = 5
    filtered_data = np.zeros_like(rectified_data)
    for i in range(num_channels):
        filtered_data[i] = low_pass_filter(rectified_data[i], fs, cutoff)
    filtered_data = filtered_data.T
    # 3. Apply normalization
    normalized_data = 2 * (filtered_data - np.min(filtered_data)) / (np.max(filtered_data) - np.min(filtered_data)) - 1

    # Interpolate data for each row
    interpolation_time = np.linspace(start_time, start_time + duration, num_samples)  # Assuming the last timestamp represents the end time
    interpolator = interp1d(timestamps, normalized_data, kind='linear', fill_value='extrapolate', axis=0)
    interpolated_data = interpolator(interpolation_time)

    return interpolated_data


# Function to process each row of the DataFrame
def process_row(row, num_samples=100, duration=10, num_clips = 1):
    # Preprocess the data
    sides = ["left", "right"]
    tot_time = row["myo_left_timestamps"][-1] - row["myo_left_timestamps"][0]
    return_rows = []
    start_time = np.linspace(0, tot_time-duration, num_clips)
    for st in start_time:
        final_data = {}
        if duration < tot_time:
            for side in sides:
                final_data[side] = preprocess_row(row, st, duration, num_samples, side)
        else:
            for side in sides:
                final_data[side] = preprocess_row(row, 0, tot_time, num_samples, side)
        final_data = np.hstack((final_data["left"], final_data["right"]))
        return_rows.append((row["description"], final_data))
    return return_rows


In [8]:
num_samples = 100
duration = 10 # Duration in seconds
num_clips = 10 

# Assuming your DataFrame is named df
df = emg_annotations[["myo_left_timestamps", "description", "myo_left_readings", "myo_right_timestamps", "myo_right_readings"]].tail(-1)
# Assuming your DataFrame is named df
processed_data_list = df.apply(process_row, axis=1, args=(num_samples, duration, num_clips))

# Flatten the list of lists into a single list of tuples
processed_data_tuples = [item for sublist in processed_data_list for item in sublist]

# Convert the list of tuples to a DataFrame
processed_data = pd.DataFrame(processed_data_tuples, columns=['description', 'myo_readings'])
print(processed_data.head())
print(processed_data.shape)
pd.to_pickle(processed_data, "processed_data.pkl")

                                         description  \
0  Get/replace items from refrigerator/cabinets/d...   
1  Get/replace items from refrigerator/cabinets/d...   
2  Get/replace items from refrigerator/cabinets/d...   
3  Get/replace items from refrigerator/cabinets/d...   
4  Get/replace items from refrigerator/cabinets/d...   

                                        myo_readings  
0  [[-0.7846153846153846, -0.8461538461538461, -4...  
1  [[-0.7846153846153846, -0.8461538461538461, -4...  
2  [[-0.7846153846153846, -0.8461538461538461, -4...  
3  [[-0.7846153846153846, -0.8461538461538461, -4...  
4  [[-0.7846153846153846, -0.8461538461538461, -4...  
(590, 2)
